In [1]:
from modules.eve import Eve
import pandas as pd
from modules.models_manager.models_manager import ModelsManager
import networkx as nx
from pyvis.network import Network

In [2]:
eve = Eve()
eve.initialize(username='farzad')

In [3]:
dfg = eve.user_manager.get_keyphrases()
raw_json=dfg.to_dict('records') # this is how the api comes

In [4]:
raw_json

[{'node_id': 1,
  'session_id': 1,
  'node': 'Minimalist Lifestyle',
  'parent_node': 'Personal Development'},
 {'node_id': 2,
  'session_id': 1,
  'node': 'Minimalist Lifestyle',
  'parent_node': 'Personal Development'},
 {'node_id': 3,
  'session_id': 1,
  'node': 'Minimalist Lifestyle',
  'parent_node': 'Personal Development'},
 {'node_id': 4,
  'session_id': 1,
  'node': 'Minimalist Lifestyle',
  'parent_node': 'Personal Development'},
 {'node_id': 5,
  'session_id': 1,
  'node': 'Minimalist Lifestyle',
  'parent_node': 'Personal Development'},
 {'node_id': 6,
  'session_id': 1,
  'node': 'Minimalist Lifestyle',
  'parent_node': 'Personal Development'},
 {'node_id': 7,
  'session_id': 1,
  'node': 'Minimalist Lifestyle',
  'parent_node': 'Personal Development'},
 {'node_id': 8,
  'session_id': 1,
  'node': 'Minimalist Lifestyle',
  'parent_node': 'Personal Development'},
 {'node_id': 9,
  'session_id': 1,
  'node': 'Minimalist Lifestyle',
  'parent_node': 'Personal Development'},
 

In [136]:
dfg.head(5)

node_id  session_id                  node           parent_node
0        1           1  Minimalist Lifestyle  Personal Development
1        2           1  Minimalist Lifestyle  Personal Development
2        3           1  Minimalist Lifestyle  Personal Development
3        4           1  Minimalist Lifestyle  Personal Development
4        5           1  Minimalist Lifestyle  Personal Development

In [159]:
bad = '"The Home & Living category focuses on aspects of managing and living in a home, from organization and cleaning to decor and maintenance. It provides information and advice on creating a comfortable, functional living environment. This can also encompass topics like gardening, DIY home projects, and sustainable living practices."'
# if bad changed to Home & Living
for row in dfg.index:
    if dfg.loc[row, 'parent_node'] == bad:
        dfg.loc[row, 'parent_node'] = 'Home & Living'

In [174]:
# group by node and parent node
dfgg = dfg.groupby(['node', 'parent_node']).size().reset_index(name='count')

In [176]:
dfgg.head(5)

node           parent_node  count
0         Achievements         Personal Growt     14
1     Anxiety and Fears         Mental Health     15
2  Budgeting and Saving               Finance      7
3    Career Progression                   Wor     14
4     Career and Financ  Personal Development     39

In [183]:
# normalize the count
dfgg['count'] = dfgg['count'] / dfgg['count'].max()

In [184]:
nodes = pd.concat([dfgg['node'], dfgg['parent_node']], axis=0).unique()
nodes.shape

(76,)

In [ ]:

G = nx.Graph()

## Add nodes to the graph
for node in nodes:
    G.add_node(
        str(node)
    )

## Add edges to the graph
for index, row in dfgg.iterrows():
    G.add_edge(
        str(row["node"]),
        str(row["parent_node"]),
        weight=row["count"]
    )

In [186]:
communities_generator = nx.community.girvan_newman(G)
top_level_communities = next(communities_generator)
next_level_communities = next(communities_generator)
communities = sorted(map(sorted, next_level_communities))
print("Number of Communities = ", len(communities))
print(communities)

Number of Communities =  20
[['Achievements ', 'Personal Growt', 'Resilience and Grit'], ['Anxiety and Fears', 'Coping Strategies', 'Dreams (Sleep)', 'Emotions', 'Health and Wellness', 'Mental Health', 'Self-Care'], ['Budgeting and Saving', 'Finance'], ['Career Development', 'Work'], ['Career Progressio', 'Innovative Thinking', 'Skills & Talents '], ['Career Progression', 'Wor'], ['Career and Financ', 'Challenges & Setbacks ', 'Daily Routines ', 'Decluttering and Organization', 'Digital Footprint', 'Eco-Friendly Living ', 'Education', 'Goals & Aspirations', 'Happiness and Fulfillment ', 'Hobbies & Interests', 'Learning and Education', 'Lifelong Learning', 'Mental Wellness ', 'Mindful Living', 'Mindfulness and Meditation ', 'Minimalist Lifestyle', 'Minimizing & Evaluating Personal Belongings', 'Personal Development', 'Personality', 'Philosophical Beliefs ', 'Physical Health ', 'Relationships ', 'Self-Improvement', 'Spiritual Life', 'Work-Life Balance'], ['Community Engagement', 'Finance

In [26]:
import seaborn as sns
import random
palette = "hls"

## Now add these colors to communities and make another dataframe
def colors2Community(communities) -> pd.DataFrame:
    ## Define a color palette
    p = sns.color_palette(palette, len(communities)).as_hex()
    random.shuffle(p)
    rows = []
    group = 0
    for community in communities:
        color = p.pop()
        group += 1
        for node in community:
            rows += [{"node": node, "color": color, "group": group}]
    df_colors = pd.DataFrame(rows)
    return df_colors

In [187]:
colors = colors2Community(communities)
colors.head(5)

node    color  group
0        Achievements   #57acdb      1
1       Personal Growt  #57acdb      1
2  Resilience and Grit  #57acdb      1
3    Anxiety and Fears  #dbae57      2
4    Coping Strategies  #dbae57      2

In [188]:
for index, row in colors.iterrows():
    G.nodes[row['node']]['group'] = row['group']
    G.nodes[row['node']]['color'] = row['color']
    G.nodes[row['node']]['size'] = G.degree[row['node']]

In [189]:
from pyvis.network import Network

graph_output_directory = "./docs/index.html"

net = Network(
    notebook=False,
    # bgcolor="#1a1a1a",
    cdn_resources="remote",
    height="900px",
    width="100%",
    select_menu=True,
    # font_color="#cccccc",
    filter_menu=False,
)

net.from_nx(G)
# net.repulsion(node_distance=150, spring_length=400)
net.force_atlas_2based(central_gravity=0.015, gravity=-31)
# net.barnes_hut(gravity=-18100, central_gravity=5.05, spring_length=380)
net.show_buttons(filter_=["physics"])

net.show('keybert.html', notebook=False)

keybert.html
